### **ENV**

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import time
from datetime import datetime
import os
from dotenv import load_dotenv

import mysql.connector

import gspread
from gspread.exceptions import APIError
from gspread.utils import rowcol_to_a1
#from gspread_formatting import *
#from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

import prettytable as pt

#from telegram import ParseMode
#from telegram.ext import CallbackContext, Updater

import logging

load_dotenv()

#asana creds
asana_token = os.getenv('ASANA_TOKEN')
workspace_gid = os.getenv('WORKSPACE_GID')
team_gid = os.getenv('TEAM_GID')

#telegram creds
bot_token = os.getenv('BOT_TOKEN')

#google sheets creds
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

#mysql creds
db_host = os.getenv("DB_HOST")
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")
database = os.getenv("DATABASE")

#skip list
with open('users_to_skip.json', 'r', encoding='utf-8') as config_file:
    config = json.load(config_file)
users_to_skip = config.get('users_to_skip', '')
users_to_skip
db_pass

'2UiB3WBuxL8wFs!'

#### users_df from GS

In [2]:
#### get last asana data on users

def get_users(asana_token, team_gid):
    
    url = f"https://app.asana.com/api/1.0/teams/{team_gid}/users"
    headers = {'Authorization': f'Bearer {asana_token}'}
    
    payload = {
    'opt_fields': 'name',
     'opt_pretty': True
     }
    
    response = requests.get(url, headers=headers, params=payload)
    users_json = response.json()
    users_df = pd.json_normalize(users_json['data'], max_level=1)
    
    rows_to_keep = []
    for _, row in users_df.iterrows():
        if row['name'] not in users_to_skip:
            rows_to_keep.append(row)
            
    users_df = pd.DataFrame(rows_to_keep).reset_index(drop=True)
    users_df = users_df.rename(columns={'gid':'user_gid'})                      
    users_df['idx'] = (users_df.index + 1).tolist()
    
    col_order = ['idx', 'user_gid', 'name']
    users_df = users_df[col_order]
    
    return users_df

asana_users = get_users(asana_token, team_gid)
asana_users

,idx,user_gid,name
0,1,1169819852543243,Burmistrov Dmitrii
1,2,1200684451347406,Elena Maeva
2,3,1206620019823578,Maxim Letunovskii
3,4,1208216811351938,sammaleen
4,5,1202838673949936,Александр Виноградов
5,6,1205394271497253,Александр Смирнов
6,7,1207898444509346,Алиса Гончарова
7,8,1207866432174028,Анастасия Романова
8,9,1206236521952192,Андрей Васильев
9,10,1201758129291395,Анна Воловикова


In [4]:
#### get current GS table

table = client.open('asana_users')
sheet = table.worksheet('users')
gs_data = sheet.get('A:D')
gs_users = pd.DataFrame(gs_data[1:], columns=gs_data[0])


# update asana_users with tokens from gs_users

def upd_gs_users(asana_users, gs_users):
    
    gs_users_upd = asana_users.copy()
    gs_users_upd['user_token'] = None
    
    for idx, row in gs_users_upd.iterrows():
        
        user = row['name']
        
        if user in gs_users['name'].values:
             gs_users_upd.loc[idx,'user_token'] = gs_users.loc[gs_users['name'] == user, 'user_token'].values[0]
        
    return gs_users_upd

gs_users_upd = upd_gs_users(asana_users, gs_users)


#### update table in GS

table = client.open('asana_users')
sheet = table.worksheet('users')
sheet.batch_clear(['A:D'])

data = [gs_users_upd.columns.values.tolist()] + gs_users_upd.values.tolist()
sheet.update(values=data, range_name='A1')


#### using gs_users_upd copy as users_df

users_df = gs_users_upd.copy()
users_df

,idx,user_gid,name,user_token
0,1,1169819852543243,Burmistrov Dmitrii,2/1169819852543243/1208838258983824:2b33dcaa4b...
1,2,1200684451347406,Elena Maeva,2/1200684451347406/1205697720563281:04f45ed138...
2,3,1206620019823578,Maxim Letunovskii,None
3,4,1208216811351938,sammaleen,2/1208216811351938/1208785586613550:92e6872421...
4,5,1202838673949936,Александр Виноградов,2/1202838673949936/1208803692653018:ada5c2e826...
5,6,1205394271497253,Александр Смирнов,2/1205394271497253/1208804850093387:b25d672b9e...
6,7,1207898444509346,Алиса Гончарова,2/1207898444509346/1208803460085212:63667c3734...
7,8,1207866432174028,Анастасия Романова,2/1207866432174028/1208804088059801:3b2dc008ce...
8,9,1206236521952192,Андрей Васильев,2/1206236521952192/1208803516086639:ed8f35591e...
9,10,1201758129291395,Анна Воловикова,None


### **TODAY TASKS**

#### users_df_upd + list_gid

In [5]:
# upd users_df with list_gid 

def get_users_list(users_df):
    
    users_df_upd = users_df.copy()
    users_df_upd['list_gid'] = None
    
    for idx, row in users_df_upd.iterrows():

        user_gid = row['user_gid']
        user_token = row['user_token']
        user = row['name']
        
        if user_token:
        
            url = f"https://app.asana.com/api/1.0/users/{user_gid}/user_task_list"
            headers = {'Authorization': f'Bearer {user_token}'}

            payload = {
                'workspace': workspace_gid,
                'opt_fields': '',
                'opt_pretty': True  
            }

            response = requests.get(url, headers=headers, params=payload)
            status = response.status_code

            if status == 200:
                response_json = response.json()
            else:
                print(f'{user} - error: {status}')
                continue

            list_gid = pd.json_normalize(response_json['data'], max_level=1)
            users_df_upd.loc[idx,'list_gid'] = list_gid.loc[0][0]
            
        else:
            print(f"{user} - no token")

    return users_df_upd

users_df_upd = get_users_list(users_df)
users_df_upd

Maxim Letunovskii - no token
Анна Воловикова - no token
Дауддин Дауди - no token
Дмитрий Лебедев - no token
Елизавета Анастасова - no token
Иван Гришин - no token
Константин Королев - no token
Никита Помолев - no token
Шлыков Дмитрий - no token


,idx,user_gid,name,user_token,list_gid
0,1,1169819852543243,Burmistrov Dmitrii,2/1169819852543243/1208838258983824:2b33dcaa4b...,1169819853809249
1,2,1200684451347406,Elena Maeva,2/1200684451347406/1205697720563281:04f45ed138...,1200684497000735
2,3,1206620019823578,Maxim Letunovskii,None,None
3,4,1208216811351938,sammaleen,2/1208216811351938/1208785586613550:92e6872421...,1208216811351969
4,5,1202838673949936,Александр Виноградов,2/1202838673949936/1208803692653018:ada5c2e826...,1202838674015997
5,6,1205394271497253,Александр Смирнов,2/1205394271497253/1208804850093387:b25d672b9e...,1205394272419322
6,7,1207898444509346,Алиса Гончарова,2/1207898444509346/1208803460085212:63667c3734...,1207898449085343
7,8,1207866432174028,Анастасия Романова,2/1207866432174028/1208804088059801:3b2dc008ce...,1207866433515798
8,9,1206236521952192,Андрей Васильев,2/1206236521952192/1208803516086639:ed8f35591e...,1206236522300401
9,10,1201758129291395,Анна Воловикова,None,None


#### my_tasks_dict 

In [6]:
#### get tasks in personal lists

def get_my_tasks(users_df_upd):
    
    my_tasks_dict = {}
    
    for _, row in users_df_upd.iterrows():
        
        list_gid = row['list_gid']
        user_token = row['user_token']
        user_gid = row['user_gid']
        user = row['name']
        
        my_tasks = []  # all json data on tasks (all pages)
        
        if list_gid:  
        
            url = f"https://app.asana.com/api/1.0/user_task_lists/{list_gid}/tasks"
            headers = {'Authorization': f'Bearer {user_token}'}

            payload = {
                'completed_since': 'now',
                'opt_fields': 'name, created_at, due_on, start_on, projects, projects.name, section.name, notes, assignee_section.name, created_by.name, created_by.gid, permalink_url',
                'limit': 100,
                'opt_pretty': True  
            }
            
            # pagination
            while True:
                response = requests.get(url, headers=headers, params=payload)
                
                if response.status_code == 200:
                    json_data = response.json()
                    
                    if json_data.get('data'): 
                        my_tasks.extend(json_data['data'])
                    
                    # check for more pages presence
                    if json_data.get('next_page'): 
                        payload['offset'] = json_data['next_page']['offset']  # update for next page
                    else:
                        break 
                else:
                    print(f"{user} / {user_gid} - error: {response.status_code}")
                    break
                    
            if my_tasks:
                my_tasks_df = pd.json_normalize(my_tasks, max_level=3) 
                my_tasks_df.rename(columns={'gid':'task_gid', 'name':'task_name','permalink_url':'url','projects':'project_name'}, inplace=True)
                #my_tasks_df = my_tasks_df.sort_values('created_at', ascending = False).reset_index(drop=True)
                
                # SECTION NAME = TODAY и СЕГОДНЯ
                #my_tasks_df = my_tasks_df[my_tasks_df['assignee_section.name'] == 'Today']
                my_tasks_df = my_tasks_df[(my_tasks_df['assignee_section.name'] == 'Today') | (my_tasks_df['assignee_section.name'] == 'Сегодня')]
                
                # extracting project names from nested list []
                if 'project_name' in my_tasks_df.columns:
                    my_tasks_df['project_name'] = my_tasks_df['project_name'].apply(
                        lambda x: x[0]['name'] if isinstance(x, list) and x else '')
                
                #re-order columns
                order = ['task_gid','project_name','task_name',
                         'start_on','due_on','notes',
                         'created_at','url','created_by.gid',
                         'created_by.name','assignee_section.gid','assignee_section.name']
                
                my_tasks_df = my_tasks_df[order]
                
            else:
                my_tasks_df = pd.DataFrame() 
            
        else:
            my_tasks_df = pd.DataFrame()  
        
        my_tasks_dict[user_gid] = my_tasks_df
        
    return my_tasks_dict

my_tasks_dict = get_my_tasks(users_df_upd)
my_tasks_dict[users_df_upd[users_df_upd['name'] == 'Elena Maeva']['user_gid'].values[0]]

,task_gid,project_name,task_name,start_on,due_on,notes,created_at,url,created_by.gid,created_by.name,assignee_section.gid,assignee_section.name
0,1208894691610594,,Проработать базу данных бота,None,2024-12-20,,2024-12-03T20:26:23.116Z,https://app.asana.com/0/1208894691610594/12088...,1200684451347406,Elena Maeva,1200684497011158,Today
1,1208846925326940,,Проработать ф-л авторизации и основную структу...,None,2024-11-29,,2024-11-26T07:42:51.511Z,https://app.asana.com/0/1208846925326940/12088...,1200684451347406,Elena Maeva,1209030033487246,Сегодня


In [14]:
my_tasks_dict[users_df_upd[users_df_upd['name'] == 'Elena Maeva']['user_gid'].values[0]].columns.tolist()

['task_gid',
 'project_name',
 'task_name',
 'start_on',
 'due_on',
 'notes',
 'created_at',
 'url',
 'created_by.gid',
 'created_by.name',
 'assignee_section.gid',
 'assignee_section.name']

In [20]:
my_tasks_dict.keys()

dict_keys(['1169819852543243', '1200684451347406', '1206620019823578', '1208216811351938', '1202838673949936', '1205394271497253', '1207898444509346', '1207866432174028', '1206236521952192', '1201758129291395', '1208448414397992', '1206632185530228', '1208879411492437', '1208557702544636', '1208448418817661', '1202140862733702', '1208945482173157', '1206644695099440', '1208796650620074', '1207297908474016', '1206039926119848'])

In [24]:
my_tasks_dict[users_df_upd[users_df_upd['name'] == 'Кирилл Матрёничев']['user_gid'].values[0]]

,task_gid,project_name,task_name,start_on,due_on,notes,created_at,url,created_by.gid,created_by.name,assignee_section.gid,assignee_section.name
0,1208983630505397,,Получить ОС по оф. письму с предложением экспл...,None,2025-01-15,,2024-12-17T15:00:46.588Z,https://app.asana.com/0/1208983630505397/12089...,1202140862733702,Кирилл Матрёничев,1202140862767680,Сегодня
1,1208995779676141,,Получить ОС от Гульназ по предоставлению образ...,None,2024-12-23,,2024-12-18T15:00:08.822Z,https://app.asana.com/0/1208995779676141/12089...,1202140862733702,Кирилл Матрёничев,1202140862767680,Сегодня
2,1208983625656249,,Получить ОС от Гульназ по обновленным схемам P&ID,None,2024-12-23,,2024-12-16T07:32:33.551Z,https://app.asana.com/0/1208983625656249/12089...,1202140862733702,Кирилл Матрёничев,1202140862767680,Сегодня
3,1208983630505419,,Направить документы о прайсе на тестирование к...,None,2024-12-25,,2024-12-18T11:32:06.049Z,https://app.asana.com/0/1208983630505419/12089...,1202140862733702,Кирилл Матрёничев,1202140862767680,Сегодня
4,1208995780784988,,Обсудить условия сотрудничества с ХромоСиб по ...,None,2024-12-23,,2024-12-19T06:06:44.387Z,https://app.asana.com/0/1208995780784988/12089...,1202140862733702,Кирилл Матрёничев,1202140862767680,Сегодня
5,1208995780784989,,"Подписать NDA, направить материалы по ПГХ в Хр...",None,2024-12-20,,2024-12-19T06:07:30.289Z,https://app.asana.com/0/1208995780784989/12089...,1202140862733702,Кирилл Матрёничев,1202140862767680,Сегодня
6,1208838269302607,,Подготовить и направить ТКП на адсорбционную у...,None,2024-12-25,,2024-11-26T11:33:58.401Z,https://app.asana.com/0/1208838269302607/12088...,1169819852543243,Burmistrov Dmitrii,1202140862767680,Сегодня
7,1208983630505439,,Подготовить материалы для аутсорсового констру...,None,2024-12-20,,2024-12-18T12:49:53.676Z,https://app.asana.com/0/1208983630505439/12089...,1202140862733702,Кирилл Матрёничев,1202140862767680,Сегодня
8,1208987898717687,СИБУР - Изготовление пилотных установок,"Связаться со специалистом ХРОМОС на ЗСНХ, дого...",None,2024-12-20,,2024-12-17T18:56:48.576Z,https://app.asana.com/0/1204847055489491/12089...,1202140862733702,Кирилл Матрёничев,1202140862767680,Сегодня
9,1208935402106186,,"Обсудить договор на этап 1, запросить данные т...",None,2024-12-18,,2024-12-09T10:48:41.228Z,https://app.asana.com/0/1208935402106186/12089...,1202140862733702,Кирилл Матрёничев,1202140862767680,Сегодня


### **LOAD TO MYSQL**

In [15]:
datetime.now()

datetime.datetime(2024, 12, 23, 10, 38, 27, 896604)

In [14]:
current_date = datetime.now()
date_extracted = current_date.strftime("%Y-%m-%d")
time_extracted = 

conn = mysql.connector.connect(user = db_user,
                               password = db_pass,
                               host = db_host,
                               database = database)
cursor = conn.cursor()

for user_gid, tasks_df in my_tasks_dict.items():
    
    user_name = users_df_upd[users_df_upd['user_gid'] == user_gid]['name'].values[0]
    
    for _, row in tasks_df.iterrows():
        
        cursor.execute("""
            INSERT INTO tasks (
                user_gid, task_gid, project_name, task_name, start_on, due_on, notes, 
                created_at, url, created_by_gid, created_by_name, assignee_section_gid, 
                assignee_section_name, data, date_extracted
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            user_gid,
            row['task_gid'],
            row['project_name'],
            row['task_name'],
            row['start_on'],
            row['due_on'],
            row['notes'],
            row['created_at'],
            row['url'],
            row['created_by.gid'],
            row['created_by.name'],
            row['assignee_section.gid'],
            row['assignee_section.name'],
            date_extracted
        ))

conn.commit()
cursor.close()
conn.close()

OperationalError: 2013 (HY000): Lost connection to MySQL server at 'reading initial communication packet', system error: 0

In [13]:
users_df_upd

,idx,user_gid,name,user_token,list_gid
0,1,1169819852543243,Burmistrov Dmitrii,2/1169819852543243/1208838258983824:2b33dcaa4b...,1169819853809249
1,2,1200684451347406,Elena Maeva,2/1200684451347406/1205697720563281:04f45ed138...,1200684497000735
2,3,1206620019823578,Maxim Letunovskii,None,None
3,4,1208216811351938,sammaleen,2/1208216811351938/1208785586613550:92e6872421...,1208216811351969
4,5,1202838673949936,Александр Виноградов,2/1202838673949936/1208803692653018:ada5c2e826...,1202838674015997
5,6,1205394271497253,Александр Смирнов,2/1205394271497253/1208804850093387:b25d672b9e...,1205394272419322
6,7,1207898444509346,Алиса Гончарова,2/1207898444509346/1208803460085212:63667c3734...,1207898449085343
7,8,1207866432174028,Анастасия Романова,2/1207866432174028/1208804088059801:3b2dc008ce...,1207866433515798
8,9,1206236521952192,Андрей Васильев,2/1206236521952192/1208803516086639:ed8f35591e...,1206236522300401
9,10,1201758129291395,Анна Воловикова,None,None


### EXP / ARCHIVE